### Formatting labels on plots.

In Lets-Plot you can apply a formatting to:

- axis break values.
- labels displayed by `geomText()`.
- tooltip text.
- facet labels.

Using format string you can format values of numeric and date-time types.

In addition, you can use a *string template*.

In *string template* the value's format string is surrounded by curly braces: `"... {.2f} ..."`.

An empty placeholder `{}` is also allowed. In this case a default string representation will be shown. This is also applicable to categorical values.

To learn more about formatting templates see: [Formatting](https://lets-plot.org/kotlin/formats.html).

In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.9.3. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.5.2.

### The US Unemployment Rates 2000-2016

In [3]:
var economics = DataFrame.readCSV("https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/economics.csv")

In [4]:
economics.head()

rownames,date,pce,pop,psavert,uempmed,unemploy
1,1967-07-01,506.700000,198712.000000,12.600000,4.500000,2944
2,1967-08-01,509.800000,198911.000000,12.600000,4.700000,2945
3,1967-09-01,515.600000,199113.000000,11.900000,4.600000,2958
4,1967-10-01,512.200000,199311.000000,12.900000,4.900000,3143
5,1967-11-01,517.400000,199498.000000,12.800000,4.700000,3066


In [5]:
import kotlinx.datetime.*

fun LocalDate.toEpochMilliseconds(): Long {
    return atStartOfDayIn((TimeZone.currentSystemDefault())).toEpochMilliseconds()
}

In [6]:
val startDate = LocalDate.parse("2000-01-01")
economics = economics.filter { date.compareTo(startDate) >= 0 }.convert {date}.with {it.toEpochMilliseconds()}

In [7]:
val p = letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
    geomLine() +
    ylab("unemployment rate") +
    ggsize(900, 400)

#### Default plot (no formatting)

#### Apply formatting to X and Y axis labels.

Use the `format` parameter in `scaleXxx()`.

Note that the text in tooltips is now also formatted.

In [8]:
p + scaleXDateTime(format="%b %Y") + scaleYContinuous(format="{} %")

<path d="M36.82164691791854 316.20000000000005 L36.82164691791854 316.20000000000005 L40.92105289703363 311.55 L44.755981071044516 313.1 L48.84987709588677 311.55 L52.81704417244964 316.20000000000005 L56.91645015156473 317.75 L60.88361722812783 313.1 L64.98302320724292 308.45000000000005 L69.08242918635801 325.5 L73.04959626292089 311.55 L77.15451219630904 311.55 L81.12167927287192 313.1 L85.221085251987 316.20000000000005 L89.3204912311021 311.55 L93.02318050256099 303.8 L97.11707652740301 314.65000000000003 L101.08424360396612 308.45000000000005 L105.1836495830812 313.1 L109.15081665964408 300.70000000000005 L113.25022263875917 299.15000000000003 L117.34962861787426 294.5 L121.31679569443736 292.95000000000005 L125.42171162782529 286.75 L129.3888787043884 279.0 L133.48828468350348 275.9 L137.58769066261857 277.45000000000005 L141.29037993407724 275.9 L145.3842759589195 268.15 L149.35144303548236 258.85 L153.45084901459745 235.60000000000002 L157.41801609116055 268.15 L161.51742207027564 266.6 L165.61682804939073 258.85 L169.5839951259536 257.30000000000007 L173.68891105934176 261.95000000000005 L177.65607813590464 257.30000000000007 L181.75548411501973 257.30000000000007 L185.85489009413482 258.85 L189.5575793655937 255.75000000000003 L193.65147539043573 248.00000000000003 L197.61864246699884 252.65 L201.71804844611393 227.85000000000002 L205.6852155226768 246.45000000000002 L209.7846215017919 249.55000000000004 L213.88402748090698 248.00000000000003 L217.85119455747008 244.9 L221.95611049085824 246.45000000000002 L225.9232775674211 244.9 L230.0226835465362 241.80000000000004 L234.1220895256513 248.00000000000003 L237.95701769966217 248.00000000000003 L242.0509137245042 258.85 L246.0180808010673 252.65 L250.1174867801824 235.60000000000002 L254.08465385674526 268.15 L258.18405983586035 263.5 L262.28346581497544 257.30000000000007 L266.25063289153854 258.85 L270.3555488249267 255.75000000000003 L274.3227159014896 258.85 L278.42212188060466 260.4 L282.52152785971975 263.5 L286.22421713117865 261.95000000000005 L290.3181131560207 266.6 L294.28528023258355 265.05000000000007 L298.38468621169864 266.6 L302.35185328826174 269.70000000000005 L306.4512592673768 263.5 L310.5506652464919 275.9 L314.5178323230548 272.80000000000007 L318.62274825644295 274.35 L322.58991533300605 271.25 L326.68932131212114 272.80000000000007 L330.7887272912362 265.05000000000007 L334.4914165626949 271.25 L338.58531258753715 275.9 L342.5524796641 274.35 L346.6518856432151 292.95000000000005 L350.6190527197782 282.1 L354.7184586988933 275.9 L358.8178646780084 282.1 L362.78503175457126 283.65000000000003 L366.8899476879594 277.45000000000005 L370.8571147645223 289.85 L374.9565207436374 277.45000000000005 L379.0559267227525 274.35 L382.75861599421137 265.05000000000007 L386.8525120190534 272.80000000000007 L390.8196790956165 279.0 L394.9190850747316 286.75 L398.88625215129446 271.25 L402.98565813040955 269.70000000000005 L407.08506410952464 271.25 L411.05223118608774 275.9 L415.15714711947567 272.80000000000007 L419.12431419603877 275.9 L423.22372017515386 266.6 L427.32312615426895 271.25 L431.15805432827983 271.25 L435.25195035312186 260.4 L439.21911742968496 283.65000000000003 L443.31852340880005 266.6 L447.2856904853629 255.75000000000003 L451.385096464478 255.75000000000003 L455.4845024435931 248.00000000000003 L459.4516695201562 244.9 L463.55658545354413 254.20000000000002 L467.52375253010723 243.35000000000002 L471.6231585092223 240.25000000000003 L475.7225644883374 224.75000000000003 L479.4252537597961 215.45000000000002 L483.51914978463833 203.05000000000004 L487.4863168612012 186.00000000000003 L491.5857228403163 139.50000000000006 L495.5528899168794 158.10000000000002 L499.6522958959945 153.45000000000002 L503.7517018751096 130.2 L507.71886895167245 113.15000000000003 L511.8237848850606 99.19999999999999 L515.7909519616237 94.55000000000001 L519.8903579407388 96.10000000000002 L523.9897639198539 97.65000000000003 L527.6924531913126 89.900000000000

#### Format axis labels for breaks specified manually


In [9]:
val breaks = listOf(
    LocalDate.parse("2001-01-01"), 
    LocalDate.parse("2016-01-01")
).map {it.toEpochMilliseconds()}

p + scaleXDateTime(format="%b %Y", breaks=breaks) + scaleYContinuous(format="{} %")

<path d="M36.82164691791854 287.99270532811613 L36.82164691791854 287.99270532811613 L40.92105289703363 283.75751848505564 L44.755981071044516 285.16924743274245 L48.84987709588677 283.75751848505564 L52.81704417244964 287.99270532811613 L56.91645015156473 289.404434275803 L60.88361722812783 285.16924743274245 L64.98302320724292 280.93406058968196 L69.08242918635801 296.4630790142372 L73.04959626292089 283.75751848505564 L77.15451219630904 283.75751848505564 L81.12167927287192 285.16924743274245 L85.221085251987 287.99270532811613 L89.3204912311021 283.75751848505564 L93.02318050256099 276.6988737466214 L97.11707652740301 286.5809763804293 L101.08424360396612 280.93406058968196 L105.1836495830812 285.16924743274245 L109.15081665964408 273.8754158512477 L113.25022263875917 272.46368690356087 L117.34962861787426 268.2285000605003 L121.31679569443736 266.8167711128135 L125.42171162782529 261.1698553220661 L129.3888787043884 254.1112105836319 L133.48828468350348 251.2877526882582 L137.58769066261857 252.69948163594503 L141.29037993407724 251.2877526882582 L145.3842759589195 244.22910794982397 L149.35144303548236 235.75873426370293 L153.45084901459745 214.58280004840026 L157.41801609116055 244.22910794982397 L161.51742207027564 242.81737900213716 L165.61682804939073 235.75873426370293 L169.5839951259536 234.3470053160161 L173.68891105934176 238.5821921590766 L177.65607813590464 234.3470053160161 L181.75548411501973 234.3470053160161 L185.85489009413482 235.75873426370293 L189.5575793655937 232.93527636832926 L193.65147539043573 225.87663162989503 L197.61864246699884 230.11181847295555 L201.71804844611393 207.52415530996603 L205.6852155226768 224.46490268220816 L209.7846215017919 227.28836057758187 L213.88402748090698 225.87663162989503 L217.85119455747008 223.05317373452132 L221.95611049085824 224.46490268220816 L225.9232775674211 223.05317373452132 L230.0226835465362 220.22971583914764 L234.1220895256513 225.87663162989503 L237.95701769966217 225.87663162989503 L242.0509137245042 235.75873426370293 L246.0180808010673 230.11181847295555 L250.1174867801824 214.58280004840026 L254.08465385674526 244.22910794982397 L258.18405983586035 239.99392110676345 L262.28346581497544 234.3470053160161 L266.25063289153854 235.75873426370293 L270.3555488249267 232.93527636832926 L274.3227159014896 235.75873426370293 L278.42212188060466 237.17046321138977 L282.52152785971975 239.99392110676345 L286.22421713117865 238.5821921590766 L290.3181131560207 242.81737900213716 L294.28528023258355 241.4056500544503 L298.38468621169864 242.81737900213716 L302.35185328826174 245.64083689751084 L306.4512592673768 239.99392110676345 L310.5506652464919 251.2877526882582 L314.5178323230548 248.46429479288452 L318.62274825644295 249.87602374057136 L322.58991533300605 247.05256584519768 L326.68932131212114 248.46429479288452 L330.7887272912362 241.4056500544503 L334.4914165626949 247.05256584519768 L338.58531258753715 251.2877526882582 L342.5524796641 249.87602374057136 L346.6518856432151 266.8167711128135 L350.6190527197782 256.9346684790056 L354.7184586988933 251.2877526882582 L358.8178646780084 256.9346684790056 L362.78503175457126 258.3463974266924 L366.8899476879594 252.69948163594503 L370.8571147645223 263.9933132174398 L374.9565207436374 252.69948163594503 L379.0559267227525 249.87602374057136 L382.75861599421137 241.4056500544503 L386.8525120190534 248.46429479288452 L390.8196790956165 254.1112105836319 L394.9190850747316 261.1698553220661 L398.88625215129446 247.05256584519768 L402.98565813040955 245.64083689751084 L407.08506410952464 247.05256584519768 L411.05223118608774 251.2877526882582 L415.15714711947567 248.46429479288452 L419.12431419603877 251.2877526882582 L423.22372017515386 242.81737900213716 L427.32312615426895 247.05256584519768 L431.15805432827983 247.05256584519768 L435.25195035312186 237.17046321138977 L439.21911742968496 258.3463974266924 L443.31852340880005 242.81737900213716 L447.2856904853629 232.93527636832926 L451.385096464478 232.935

#### Configure tooltip's text and location.

In [10]:
letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
 geomLine(tooltips=layerTooltips()
        .line("Unemployment rate:|^y")
        .anchor("top_center")
        .minWidth(170)) + 
 scaleXDateTime(format="%b %Y") +
 scaleYContinuous(format="{} %") +
 ylab("unemployment rate") +
 ggsize(900, 400)

<path d="M36.82164691791854 316.20000000000005 L36.82164691791854 316.20000000000005 L40.92105289703363 311.55 L44.755981071044516 313.1 L48.84987709588677 311.55 L52.81704417244964 316.20000000000005 L56.91645015156473 317.75 L60.88361722812783 313.1 L64.98302320724292 308.45000000000005 L69.08242918635801 325.5 L73.04959626292089 311.55 L77.15451219630904 311.55 L81.12167927287192 313.1 L85.221085251987 316.20000000000005 L89.3204912311021 311.55 L93.02318050256099 303.8 L97.11707652740301 314.65000000000003 L101.08424360396612 308.45000000000005 L105.1836495830812 313.1 L109.15081665964408 300.70000000000005 L113.25022263875917 299.15000000000003 L117.34962861787426 294.5 L121.31679569443736 292.95000000000005 L125.42171162782529 286.75 L129.3888787043884 279.0 L133.48828468350348 275.9 L137.58769066261857 277.45000000000005 L141.29037993407724 275.9 L145.3842759589195 268.15 L149.35144303548236 258.85 L153.45084901459745 235.60000000000002 L157.41801609116055 268.15 L161.51742207027564 266.6 L165.61682804939073 258.85 L169.5839951259536 257.30000000000007 L173.68891105934176 261.95000000000005 L177.65607813590464 257.30000000000007 L181.75548411501973 257.30000000000007 L185.85489009413482 258.85 L189.5575793655937 255.75000000000003 L193.65147539043573 248.00000000000003 L197.61864246699884 252.65 L201.71804844611393 227.85000000000002 L205.6852155226768 246.45000000000002 L209.7846215017919 249.55000000000004 L213.88402748090698 248.00000000000003 L217.85119455747008 244.9 L221.95611049085824 246.45000000000002 L225.9232775674211 244.9 L230.0226835465362 241.80000000000004 L234.1220895256513 248.00000000000003 L237.95701769966217 248.00000000000003 L242.0509137245042 258.85 L246.0180808010673 252.65 L250.1174867801824 235.60000000000002 L254.08465385674526 268.15 L258.18405983586035 263.5 L262.28346581497544 257.30000000000007 L266.25063289153854 258.85 L270.3555488249267 255.75000000000003 L274.3227159014896 258.85 L278.42212188060466 260.4 L282.52152785971975 263.5 L286.22421713117865 261.95000000000005 L290.3181131560207 266.6 L294.28528023258355 265.05000000000007 L298.38468621169864 266.6 L302.35185328826174 269.70000000000005 L306.4512592673768 263.5 L310.5506652464919 275.9 L314.5178323230548 272.80000000000007 L318.62274825644295 274.35 L322.58991533300605 271.25 L326.68932131212114 272.80000000000007 L330.7887272912362 265.05000000000007 L334.4914165626949 271.25 L338.58531258753715 275.9 L342.5524796641 274.35 L346.6518856432151 292.95000000000005 L350.6190527197782 282.1 L354.7184586988933 275.9 L358.8178646780084 282.1 L362.78503175457126 283.65000000000003 L366.8899476879594 277.45000000000005 L370.8571147645223 289.85 L374.9565207436374 277.45000000000005 L379.0559267227525 274.35 L382.75861599421137 265.05000000000007 L386.8525120190534 272.80000000000007 L390.8196790956165 279.0 L394.9190850747316 286.75 L398.88625215129446 271.25 L402.98565813040955 269.70000000000005 L407.08506410952464 271.25 L411.05223118608774 275.9 L415.15714711947567 272.80000000000007 L419.12431419603877 275.9 L423.22372017515386 266.6 L427.32312615426895 271.25 L431.15805432827983 271.25 L435.25195035312186 260.4 L439.21911742968496 283.65000000000003 L443.31852340880005 266.6 L447.2856904853629 255.75000000000003 L451.385096464478 255.75000000000003 L455.4845024435931 248.00000000000003 L459.4516695201562 244.9 L463.55658545354413 254.20000000000002 L467.52375253010723 243.35000000000002 L471.6231585092223 240.25000000000003 L475.7225644883374 224.75000000000003 L479.4252537597961 215.45000000000002 L483.51914978463833 203.05000000000004 L487.4863168612012 186.00000000000003 L491.5857228403163 139.50000000000006 L495.5528899168794 158.10000000000002 L499.6522958959945 153.45000000000002 L503.7517018751096 130.2 L507.71886895167245 113.15000000000003 L511.8237848850606 99.19999999999999 L515.7909519616237 94.55000000000001 L519.8903579407388 96.10000000000002 L523.9897639198539 97.65000000000003 L527.6924531913126 89.900000000000

#### Format value shown in tooltip.

In [11]:
letsPlot(economics.toMap()){x="date"; y="uempmed"} +
 geomLine(tooltips=layerTooltips()
               .line("@uempmed % in @date")
               .format("date", "%B %Y")
               .anchor("top_left")
               .minWidth(170)) +
 scaleXDateTime() +
 scaleYContinuous() +
 ylab("unemployment rate") +
 ggsize(900, 400)

<path d="M38.27449041313935 316.20000000000005 L38.27449041313935 316.20000000000005 L42.535643511396074 311.55 L46.52188350653955 313.1 L50.777309248481515 311.55 L54.90100579518162 316.20000000000005 L59.162158893438345 317.75 L63.28585544013845 313.1 L67.5470085383954 308.45000000000005 L71.80816163665213 325.5 L75.93185818335223 311.55 L80.19873863792395 311.55 L84.32243518462406 313.1 L88.58358828288078 316.20000000000005 L92.84474138113751 311.55 L96.69352482472436 303.8 L100.94895056666633 314.65000000000003 L105.07264711336643 308.45000000000005 L109.33380021162316 313.1 L113.45749675832326 300.70000000000005 L117.71864985658021 299.15000000000003 L121.97980295483694 294.5 L126.10349950153704 292.95000000000005 L130.37037995610876 286.75 L134.49407650280887 279.0 L138.7552296010656 275.9 L143.01638269932255 277.45000000000005 L146.86516614290917 275.9 L151.12059188485114 268.15 L155.24428843155124 258.85 L159.5054415298082 235.60000000000002 L163.6291380765083 268.15 L167.89029117476503 266.6 L172.15144427302175 258.85 L176.27514081972186 257.30000000000007 L180.54202127429357 261.95000000000005 L184.66571782099368 257.30000000000007 L188.92687091925063 257.30000000000007 L193.18802401750736 258.85 L197.0368074610942 255.75000000000003 L201.29223320303595 248.00000000000003 L205.41592974973605 252.65 L209.677082847993 227.85000000000002 L213.8007793946931 246.45000000000002 L218.06193249294984 249.55000000000004 L222.3230855912068 248.00000000000003 L226.4467821379069 244.9 L230.71366259247839 246.45000000000002 L234.8373591391785 244.9 L239.09851223743544 241.80000000000004 L243.35966533569217 248.00000000000003 L247.34590533083565 248.00000000000003 L251.6013310727776 258.85 L255.72502761947771 252.65 L259.98618071773444 235.60000000000002 L264.10987726443454 268.15 L268.3710303626915 263.5 L272.6321834609482 257.30000000000007 L276.75588000764833 258.85 L281.02276046222005 255.75000000000003 L285.14645700892015 258.85 L289.4076101071769 260.4 L293.6687632054336 263.5 L297.51754664902046 261.95000000000005 L301.7729723909624 266.6 L305.8966689376625 265.05000000000007 L310.15782203591925 266.6 L314.2815185826196 269.70000000000005 L318.5426716808763 263.5 L322.80382477913304 275.9 L326.92752132583314 272.80000000000007 L331.19440178040486 274.35 L335.31809832710496 271.25 L339.5792514253617 272.80000000000007 L343.84040452361864 265.05000000000007 L347.68918796720527 271.25 L351.94461370914723 275.9 L356.06831025584734 274.35 L360.3294633541043 292.95000000000005 L364.4531599008044 282.1 L368.7143129990611 275.9 L372.97546609731785 282.1 L377.09916264401795 283.65000000000003 L381.36604309858967 277.45000000000005 L385.4897396452898 289.85 L389.75089274354673 277.45000000000005 L394.01204584180346 274.35 L397.8608292853903 265.05000000000007 L402.11625502733204 272.80000000000007 L406.2399515740324 279.0 L410.5011046722891 286.75 L414.6248012189892 271.25 L418.88595431724593 269.70000000000005 L423.1471074155029 271.25 L427.270803962203 275.9 L431.5376844167745 272.80000000000007 L435.6613809634748 275.9 L439.92253406173154 266.6 L444.18368715998827 271.25 L448.16992715513175 271.25 L452.4253528970737 260.4 L456.5490494437738 283.65000000000003 L460.81020254203054 266.6 L464.93389908873064 255.75000000000003 L469.1950521869876 255.75000000000003 L473.4562052852443 248.00000000000003 L477.5799018319444 244.9 L481.84678228651615 254.20000000000002 L485.97047883321625 243.35000000000002 L490.231631931473 240.25000000000003 L494.49278502972993 224.75000000000003 L498.34156847331656 215.45000000000002 L502.5969942152585 203.05000000000004 L506.7206907619586 186.00000000000003 L510.9818438602156 139.50000000000006 L515.1055404069157 158.10000000000002 L519.3666935051724 153.45000000000002 L523.6278466034291 130.2 L527.7515431501292 113.15000000000003 L532.018423604701 99.19999999999999 L536.1421201514011 94.55000000000001 L540.4032732496578 96.10000000000002 L544.6644263479147 97.65000000000003 L548.5132097915014 89.900000

#### Add the unemployment rate mean.

The `geomText` label is formatted using the `labelFormat` parameter.

In [12]:
val unemploymentMean = economics.uempmed.mean()


letsPlot(economics.toMap()){x="date"; y="uempmed"} + 
 geomLine(tooltips=layerTooltips()
        .line("Unemployment rate:|^y %")
        .anchor("top_center")
        .minWidth(170)) + 
 geomHLine(yintercept=unemploymentMean, color="red", linetype="dashed", tooltips=tooltipsNone) +
 geomText(label=unemploymentMean.toString(), 
           labelFormat="{.2f} %",
           x=startDate.toEpochMilliseconds(), y=unemploymentMean!! + 0.5, 
           color="red") +
 scaleXDateTime(format="%b %Y") + 
 scaleYContinuous(format="{} %") + 
 ylab("unemployment rate") +
 ggtitle("The US Unemployment Rates 2000-2016.") +
 ggsize(900, 400)

<path d="M36.82164691791854 297.6545454545455 L36.82164691791854 297.6545454545455 L40.92105289703363 293.27727272727276 L44.755981071044516 294.73636363636365 L48.84987709588677 293.27727272727276 L52.81704417244964 297.6545454545455 L56.91645015156473 299.1136363636364 L60.88361722812783 294.73636363636365 L64.98302320724292 290.3590909090909 L69.08242918635801 306.40909090909093 L73.04959626292089 293.27727272727276 L77.15451219630904 293.27727272727276 L81.12167927287192 294.73636363636365 L85.221085251987 297.6545454545455 L89.3204912311021 293.27727272727276 L93.02318050256099 285.9818181818182 L97.11707652740301 296.1954545454546 L101.08424360396612 290.3590909090909 L105.1836495830812 294.73636363636365 L109.15081665964408 283.0636363636364 L113.25022263875917 281.6045454545455 L117.34962861787426 277.22727272727275 L121.31679569443736 275.76818181818186 L125.42171162782529 269.9318181818182 L129.3888787043884 262.6363636363637 L133.48828468350348 259.71818181818185 L137.58769066261857 261.17727272727274 L141.29037993407724 259.71818181818185 L145.3842759589195 252.4227272727273 L149.35144303548236 243.66818181818184 L153.45084901459745 221.7818181818182 L157.41801609116055 252.4227272727273 L161.51742207027564 250.9636363636364 L165.61682804939073 243.66818181818184 L169.5839951259536 242.20909090909095 L173.68891105934176 246.58636363636364 L177.65607813590464 242.20909090909095 L181.75548411501973 242.20909090909095 L185.85489009413482 243.66818181818184 L189.5575793655937 240.75000000000003 L193.65147539043573 233.4545454545455 L197.61864246699884 237.8318181818182 L201.71804844611393 214.48636363636365 L205.6852155226768 231.99545454545455 L209.7846215017919 234.91363636363639 L213.88402748090698 233.4545454545455 L217.85119455747008 230.53636363636366 L221.95611049085824 231.99545454545455 L225.9232775674211 230.53636363636366 L230.0226835465362 227.61818181818185 L234.1220895256513 233.4545454545455 L237.95701769966217 233.4545454545455 L242.0509137245042 243.66818181818184 L246.0180808010673 237.8318181818182 L250.1174867801824 221.7818181818182 L254.08465385674526 252.4227272727273 L258.18405983586035 248.0454545454546 L262.28346581497544 242.20909090909095 L266.25063289153854 243.66818181818184 L270.3555488249267 240.75000000000003 L274.3227159014896 243.66818181818184 L278.42212188060466 245.12727272727275 L282.52152785971975 248.0454545454546 L286.22421713117865 246.58636363636364 L290.3181131560207 250.9636363636364 L294.28528023258355 249.50454545454548 L298.38468621169864 250.9636363636364 L302.35185328826174 253.8818181818182 L306.4512592673768 248.0454545454546 L310.5506652464919 259.71818181818185 L314.5178323230548 256.8 L318.62274825644295 258.2590909090909 L322.58991533300605 255.34090909090912 L326.68932131212114 256.8 L330.7887272912362 249.50454545454548 L334.4914165626949 255.34090909090912 L338.58531258753715 259.71818181818185 L342.5524796641 258.2590909090909 L346.6518856432151 275.76818181818186 L350.6190527197782 265.55454545454546 L354.7184586988933 259.71818181818185 L358.8178646780084 265.55454545454546 L362.78503175457126 267.01363636363635 L366.8899476879594 261.17727272727274 L370.8571147645223 272.85 L374.9565207436374 261.17727272727274 L379.0559267227525 258.2590909090909 L382.75861599421137 249.50454545454548 L386.8525120190534 256.8 L390.8196790956165 262.6363636363637 L394.9190850747316 269.9318181818182 L398.88625215129446 255.34090909090912 L402.98565813040955 253.8818181818182 L407.08506410952464 255.34090909090912 L411.05223118608774 259.71818181818185 L415.15714711947567 256.8 L419.12431419603877 259.71818181818185 L423.22372017515386 250.9636363636364 L427.32312615426895 255.34090909090912 L431.15805432827983 255.34090909090912 L435.25195035312186 245.12727272727275 L439.21911742968496 267.01363636363635 L443.31852340880005 250.9636363636364 L447.2856904853629 240.75000000000003 L451.385096464478 240.75000000000003 L455.4845024435931 233.4545454545455 L459.4516695201562 2